In [25]:
import numpy as np
import pandas as pd
import random

In [26]:
data = pd.read_csv('../scrapers/pfaf/all_plants.csv')

In [27]:
data

,Family,Genus,Species,CommonName,GrowthRate,HardinessZones,Height,Width,Type,Pollinators,...,Soils,pH,Preferences,Tolerances,Habitat,HabitatRange,Edibility,Medicinal,OtherUses,PFAF
0,Aceraceae,Acer,saccharinum,"Silver Maple, River Maple, Soft Maple",fast,"[3, 4, 5, 6, 7, 8, 9]",30.0,15.0,deciduous,wind,...,"['light', 'medium', 'heavy', 'heavy']","mildly acid, neutral and basic (mildly alkaline)",[],['atmospheric pollution'],"Banks of rivers, usually in sandy soils[43, 82...","Eastern N. America - New Brunswick to Florida,...",(3 of 5),(1 of 5),(4 of 5),https://pfaf.org/user/Plant.aspx?LatinName=Ace...
1,Aceraceae,Acer,saccharum,"Sugar Maple, Florida Maple, Hard Maple, Rock M...",slow,"[4, 5, 6, 7, 8]",30.0,12.0,deciduous,insects,...,"['light', 'medium', 'heavy', 'heavy']","mildly acid, neutral and basic (mildly alkalin...",[' well-drained soil'],['atmospheric pollution'],"Found in a variety of soil types, doing best i...","Eastern N. America - Newfoundland to Georgia, ...",(4 of 5),(2 of 5),(4 of 5),https://pfaf.org/user/Plant.aspx?LatinName=Ace...
2,Asteraceae or Compositae,Achillea,millefolium,"Yarrow, Boreal yarrow, California yarrow, Gian...",fast,"[4, 5, 6, 7, 8]",0.6,0.6,NaN,insects,...,"['light', 'medium', 'heavy']","mildly acid, neutral and basic (mildly alkalin...","[' well-drained soil', 'dry or moist soil']","['drought', 'maritime exposure']","Meadows, pastures, lawns etc. on all but the p...","Europe, including Britain, north to 71°, and e...",(3 of 5),(4 of 5),(4 of 5),https://pfaf.org/user/Plant.aspx?LatinName=Ach...
3,Asteraceae or Compositae,Achillea,ptarmica,"Sneeze-Wort, Sneezeweed",fast,"[3, 4, 5, 6, 7, 8, 9]",0.6,0.6,NaN,"bees, flies",...,"['light', 'medium', 'heavy']","mildly acid, neutral and basic (mildly alkaline)",[' well-drained soil'],['maritime exposure'],"Damp meadows, marshes and by streams[17].","Europe, including Britain but excluding the Me...",(2 of 5),(1 of 5),(2 of 5),https://pfaf.org/user/Plant.aspx?LatinName=Ach...
4,Actinidiaceae,Actinidia,arguta,Tara Vine,NaN,"[4, 5, 6, 7, 8]",15.0,NaN,deciduous,"bees, insects",...,"['light', 'medium', 'heavy']","mildly acid, neutral and basic (mildly alkaline)",[],[],"Climbing up trees in woodland, mountain forest...","E. Asia - China, Japan",(5 of 5),(0 of 5),(2 of 5),https://pfaf.org/user/Plant.aspx?LatinName=Act...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
429,Agavaceae,Yucca,baccata,"Spanish Bayonet, Banana yucca, Blue Yucca, Spa...",slow,"[6, 7, 8, 9, 10, 11]",0.9,NaN,evergreen,hand,...,"['light', 'medium', 'heavy']","mildly acid, neutral and basic (mildly alkaline)","[' well-drained soil', 'dry or moist soil']",['drought'],"Rocky slopes, pinyon, oak, and juniper woodlan...","South-western N. America - Colorado to Texas, ...",(4 of 5),(1 of 5),(5 of 5),https://pfaf.org/user/Plant.aspx?LatinName=Yuc...
430,Rutaceae,Zanthoxylum,americanum,"Prickly Ash - Northern, Common pricklyash, Nor...",medium,"[3, 4, 5, 6, 7]",4.0,4.0,deciduous,NaN,...,"['light', 'medium', 'heavy']","mildly acid, neutral and basic (mildly alkaline)",[' well-drained soil. Suitable pH: mildly acid...,[],Found on upland rocky hillsides and on moist l...,"Eastern N. America - Quebec to Florida, west t...",(2 of 5),(3 of 5),(2 of 5),https://pfaf.org/user/Plant.aspx?LatinName=Zan...
431,Rutaceae,Zanthoxylum,piperitum,Japanese Pepper Tree,NaN,"[5, 6, 7, 8, 9]",2.0,2.0,deciduous,NaN,...,"['light', 'medium', 'heavy']","mildly acid, neutral and basic (mildly alkaline)",[' well-drained soil. Suitable pH: mildly acid...,[],Scrub and hedges in hills and mountains in Jap...,"E. Asia - N. China, Japan, Korea.",(3 of 5),(2 of 5),(1 of 5),https://pfaf.org/user/Plant.aspx?LatinName=Zan...
432,Apiaceae or Umbelliferae,Zizia,aurea,"Golden Alexanders, Golden zizia",NaN,"[3, 4, 5, 6, 7]",0.8,NaN,NaN,insects,...,"['light', 'medium', 'heavy']","mildly acid, neutral and basic (mildly alkaline)",[],[],Moist meadowland[200]. Dry woods and rocky out...,Eastern N

In [28]:
working_data = data[['CommonName', 'GrowthRate', 'Height', 'Width', 'pH', 'Habitat']]

In [54]:
def call_plant(df, search_name):
    # dataframe with all results
    data = df[df['CommonName'].str.lower().str.contains(search_name.lower().replace(" ", "|"),na=False)]
    
    # return first result for simplicity here, later allow for user to select from results
    try:
        first_result = data.iloc[0, :]
        # get attributes, so as to not return a dataframe
        common_name = first_result['CommonName']
        height = first_result['Height']
        width = first_result['Width']
        ph = first_result['pH']
        habitat = first_result['Habitat']
        return common_name, height, width # skip the others for now
    except:
        pass


In [62]:
search_result = call_plant(df=working_data, search_name='walnut')

In [63]:
search_result
# Need to remove pfaf references in square brackets []

('Black Walnut', 30.0, 20.0)

In [57]:
class MyDesign():
    def __init__(self, length, width, units):
        self.length = length
        self.width = width
        self.units = units
        self.area = length * width
        self.overlap = 0
        # I'll use a list to store data, create dataframe when needed
        self.data = []
        
    def add_plant(self, common_name, x_pos, y_pos, height, width):
        self.data.append({
            'common_name': common_name, # should change to latin name as unique id
            'x, y': (x_pos, y_pos),
            'height': height,
            'width': width
        })
    
    def to_dataframe(self):
        df = pd.DataFrame(self.data)
        # Use groupby latin_name to sum plants 
        ### *** THIS DOESNT WORK. NEED TO MAINTAIN INDIVIDUAL X- Y- POS AND COUNT TOTALS *** ### 
        return df.groupby('common_name').agg({
            'common_name': 'count', 'x, y': 'unique', 'height': 'mean', 'width': 'mean'
        })

In [58]:
design1 = MyDesign(10, 20, 'meters')

In [64]:
plant_list = ['walnut', 'onion', 'wild garlic',
              'Rosemary', 'garlic', 'Grape', 
              'English walnut', 'english walnut', 'wild garlic']

In [67]:
for plant in plant_list:
    try:
        name, height, width = call_plant(working_data, plant)
        x = round(random.random() * design1.width, 2)
        y = round(random.random() * design1.length, 2)
        design1.add_plant(common_name=name, x_pos=x, y_pos=y, height=height, width=width)
    except TypeError:
        pass

In [68]:
design1.data

[{'common_name': 'Black Chokeberry, Black Berried Aronia',
  'x, y': (8.59, 2.65),
  'height': 2.5,
  'width': 3.0},
 {'common_name': 'Potato Onion',
  'x, y': (0.74, 5.63),
  'height': 1.2,
  'width': nan},
 {'common_name': 'Wild Leek, Broadleaf wild leek',
  'x, y': (18.12, 6.21),
  'height': 1.8,
  'width': 0.1},
 {'common_name': 'Black Walnut',
  'x, y': (18.4, 2.74),
  'height': 30.0,
  'width': 20.0},
 {'common_name': 'Potato Onion',
  'x, y': (17.01, 0.31),
  'height': 1.2,
  'width': nan},
 {'common_name': 'Wild Leek, Broadleaf wild leek',
  'x, y': (12.48, 1.18),
  'height': 1.8,
  'width': 0.1},
 {'common_name': 'Black Walnut',
  'x, y': (11.6, 9.08),
  'height': 30.0,
  'width': 20.0},
 {'common_name': 'Potato Onion',
  'x, y': (16.99, 1.31),
  'height': 1.2,
  'width': nan},
 {'common_name': 'Wild Leek, Broadleaf wild leek',
  'x, y': (17.04, 2.85),
  'height': 1.8,
  'width': 0.1},
 {'common_name': 'Canadian Garlic, Meadow garlic, Fraser meadow garlic,  Hyacinth meadow gar

In [69]:
design1_df = design1.to_dataframe()

In [70]:
design1_df

,common_name,"x, y",height,width
common_name,,,,
"Black Chokeberry, Black Berried Aronia",1,"[(8.59, 2.65)]",2.5,3.0
Black Walnut,2,"[(18.4, 2.74), (11.6, 9.08)]",30.0,20.0
"Canadian Garlic, Meadow garlic, Fraser meadow garlic, Hyacinth meadow garlic",1,"[(10.73, 5.03)]",0.5,0.2
"Daisy, Lawndaisy, English Daisy",2,"[(0.75, 8.59), (11.67, 5.87)]",0.2,0.2
"Oregon Grape, Hollyleaved barberry, Oregon Holly Grape, Oregon Holly",1,"[(12.15, 2.02)]",2.0,1.5
Potato Onion,3,"[(0.74, 5.63), (17.01, 0.31), (16.99, 1.31)]",1.2,NaN
"Wild Leek, Broadleaf wild leek",4,"[(18.12, 6.21), (12.48, 1.18), (17.04, 2.85), ...",1.8,0.1
